In [1]:
#Загружаем бибилиотеку pandas
import pandas as pd

In [8]:
#ЗАДАНИЕ 1

#Читаем файлы со списком фильмов и рейтингами
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

#Функция, которая классифицирует фильмы
#Принимает на вход строки из списка фильмов и весь датафрейм с рейтингами
#Возвращает классификацию фильма по рейтингу
def classify(movies_row, ratings_all):
    #Отбираем все значения рейтинга для фильма из текущей строки и находим среднее значение рейтинга
    rating_mean = ratings_all[ratings_all['movieId'] == movies_row['movieId']]['rating'].mean()
    #Проверяем значение рейтинга по условию задачи
    #Оценка 2 и меньше - низкий рейтинг
    if rating_mean <= 2:
        return 'Низкий рейтинг'
    #Оценка 4 и меньше - средний рейтинг
    elif 2 < rating_mean <= 4:
        return 'Средний рейтинг'
    #Оценка 4.5 и 5 - высокий рейтинг
    else:
        return 'Высокий рейтинг'
    
#Проверка результата
movies['class'] = movies.apply(classify, ratings_all = ratings, axis = 1)
movies

,movieId,title,genres,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Средний рейтинг
1,2,Jumanji (1995),Adventure|Children|Fantasy,Средний рейтинг
2,3,Grumpier Old Men (1995),Comedy|Romance,Средний рейтинг
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Средний рейтинг
4,5,Father of the Bride Part II (1995),Comedy,Средний рейтинг
...,...,...,...,...
9120,162672,Mohenjo Daro (2016),Adventure|Drama|Romance,Средний рейтинг
9121,163056,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi,Высокий рейтинг
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary,Высокий рейтинг
9123,164977,The Gay Desperado (1936),Comedy,Высокий рейтинг


In [23]:
#ЗАДАНИЕ 2

#Читаем файл со списком поисковых запросов
keywords = pd.read_csv('keywords.csv')

#Словарь с городами в регионах
geo_data = {'Центр': ['москва', 'тула', 'ярославль'],
            'Северо-Запад': ['петербург', 'псков', 'мурманск'],
            'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
            }
#Функция, которая сопоставляет запросам географическую принадлежность определенному региону
#Принимает на вход строки из списка запросов и словарь с городами в регионах
#Возвращает принадлежность региону
def geoloc(keywords_row, geo_data):
    #Разбиваем поисковый запрос на отдельные слова и переводим в нижний регистр
    words = keywords_row['keyword'].lower().split(' ')
    #Перебираем отдельные слова запроса и проверяем принадлежность к региону
    for word in words:
        if word in geo_data['Центр']:
            return 'Центр'
        if word in geo_data['Северо-Запад']:
            return 'Северо-Запад'
        if word in geo_data['Дальний Восток']:
            return 'Дальний Восток'
    #Если ни одно слово в запросе не совпадает с содержимым geo_data
    return 'undefined'
    
#Проверка результата
keywords['region'] = keywords.apply(geoloc, geo_data = geo_data, axis = 1)
keywords

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined
...,...,...,...
99995,эльдорадо старый оскол,3705,undefined
99996,frigate для firefox,3630,undefined
99997,укрсиб,3630,undefined
99998,погода в ялте на неделю,3688,undefined


In [19]:
#ЗАДАНИЕ 3 

#Загружаем модуль для работы с регулярными выражениями
import re

#Читаем файлы со списком фильмов и рейтингами
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

#Список из всех годов с 1950 по 2010
years = range(1950, 2011)

#Функция определяет год выпуска фильма
#Принимает на вход строки из списка фильмов и список годов
#Возвращает год выпуска фильма
def production_year(movies_row, years_list):
    #Ищем в названии фильма группу из 4х цифр после скобки - это год
    try:
        year = int(re.search(r'[(](\d{4})', movies_row['title']).group(1))
        #Если найденный год есть в списке годов, то возвращаем его
        if year in years_list:
            return year
        #А если нет, то возвращаем 1900 (по условиям задачи)
        else:
            return 1900
    #А если найти не получилось, то записываем 1900
    except:
        return 1900

#Применяем функцию и создаём новый столбец 'year'
movies['year'] = movies.apply(production_year, years_list = years, axis = 1)
   
#Функция считает средний рейтинг фильмов для каждого года
#Принимает на вход сгрупированные по годам строки из списка фильмов и датафрейм с рейтингом всех фильмов
#Возвращает средний рейтинг
def rating_year(movies_grouped_by_year, ratings_all):
    #В датафрейме с рейтингами всех фильмов отфильтруем строки, соответствующие movieId фильмов по годам
    result = ratings_all[ratings_all['movieId'].isin(movies_grouped_by_year['movieId'])]
    #Из полученного результата берём среднее арифметическое рейтинго фильмов 
    return result['rating'].mean()

#Группируем по годам фильмы и получаем средний рейтинг по годам
year_rating = movies.groupby('year').apply(rating_year, ratings_all = ratings)
#Теперь сортируем по рейтингу в порядке убывания
year_rating.sort_values(ascending=False)

year
1957    4.014241
1972    4.011136
1952    4.000000
1954    3.994220
1951    3.983539
          ...   
2005    3.448434
2003    3.444777
1996    3.426600
1997    3.415764
2000    3.355945
Length: 62, dtype: float64